In [ ]:
#default_exp data.load

In [ ]:
# do something like this - copy over from fastai2 walkthru 
# !nvidia-smi -L

In [ ]:
# make your Google drive accessible 
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'fastai2_library/course-v4/'

# navigate to the notebooks directory for dl2
import os
os.chdir(base_dir)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
!pwd
# cd to base_dir if above os.chdir does not work using below command
# %cd "/content/gdrive/My Drive/fastai/course-v4/"

/content/gdrive/My Drive/fastai2_library/course-v4


In [ ]:
!pip install -q -r requirements.txt

     |████████████████████████████████| 194kB 3.3MB/s 
     |████████████████████████████████| 51kB 4.3MB/s 
     |████████████████████████████████| 51kB 4.1MB/s 
     |████████████████████████████████| 1.1MB 10.9MB/s 
     |████████████████████████████████| 40kB 3.0MB/s 
     |████████████████████████████████| 92kB 7.6MB/s 
     |████████████████████████████████| 61kB 4.8MB/s 
     |████████████████████████████████| 51kB 5.0MB/s 
     |████████████████████████████████| 2.7MB 20.2MB/s 


In [ ]:
%cd nbs

/content/gdrive/My Drive/fastai2_library/course-v4/nbs


In [ ]:
#export
from fastai2.torch_basics import *

from torch.utils.data.dataloader import _MultiProcessingDataLoaderIter,_SingleProcessDataLoaderIter,_DatasetKind
_loaders = (_MultiProcessingDataLoaderIter,_SingleProcessDataLoaderIter)

In [ ]:
from nbdev.showdoc import *

In [ ]:
bs = 4
letters = list(string.ascii_lowercase)

## DataLoader

In [ ]:
#export
def _wif(worker_id):
    set_num_threads(1)
    # call PyTorch get_worker_info function
    # tells us which dataset this process is working with
    # info.dataset.d
    info = get_worker_info()
    ds = info.dataset.d
    # tells us how many workers are there in info.num_workers
    # id of this worker into that list of workers avail in info.id
    # Use this info to parallelize things automatically 
    ds.nw,ds.offs = info.num_workers,info.id
    # each worker gets a separate random seed to prevent horrible things
    # from happening with random seeds being duplicated
    set_seed(info.seed)
    ds.wif()

class _FakeLoader:
    # PyTorch expects an object with this list with expected values, used
    # below in DataLoader __iter__
    _IterableDataset_len_called,_auto_collation,collate_fn,drop_last,dataset_kind,_dataset_kind,_index_sampler = (
        None,False,noops,False,_DatasetKind.Iterable,_DatasetKind.Iterable,Inf.count)
    def __init__(self, d, pin_memory, num_workers, timeout):
        # Note PyTorch (1.2+ versions?) added worker_init_fn 
        # which is a callback in PyTorch dataloader infrastructure which is 
        # called everytime a NEW PROCESS is fired off.
        # fastai has a worker information function _wif which does 
        # things mentioned above in _wif
        self.dataset,self.default,self.worker_init_fn = self,d,_wif
        store_attr(self, 'd,pin_memory,num_workers,timeout')

    # Calling this is what passes it back to fastai dataloader
    def __iter__(self): return iter(self.d.create_batches(self.d.sample()))

    @property
    def multiprocessing_context(self): return (None,multiprocessing)[self.num_workers>0]

    @contextmanager
    def no_multiproc(self):
        old_nw = self.num_workers
        try:
            self.num_workers = 0
            yield self.d
        finally: self.num_workers = old_nw

_collate_types = (ndarray, Tensor, typing.Mapping, str)

In [ ]:
#export
def fa_collate(t):
    b = t[0]
    # Pytorch by default calls default_collate to take all items in your datasets
    # turn them into a batch if you have a batch size or use default_convert
    # fastai_collate handles a few types that PyTorch collate does not handle
    # does not change behavior but more things should work correctly
    # used in create_batch
    return (default_collate(t) if isinstance(b, _collate_types)
            else type(t[0])([fa_collate(s) for s in zip(*t)]) if isinstance(b, Sequence)
            else default_collate(t))

In [ ]:
#e.g. x is int, y is tuple
t = [(1,(2,3)),(1,(2,3))]
test_eq(fa_collate(t), default_collate(t))
test_eq(L(fa_collate(t)).map(type), [Tensor,tuple])

t = [(1,(2,(3,4))),(1,(2,(3,4)))]
test_eq(fa_collate(t), default_collate(t))
test_eq(L(fa_collate(t)).map(type), [Tensor,tuple])
test_eq(L(fa_collate(t)[1]).map(type), [Tensor,tuple])

In [ ]:
#export
def fa_convert(t):
    # handles a few types that PyTorch convert does not handle
    # does not change behavior but more things should work correctly
    # used in create_batch
    return (default_convert(t) if isinstance(t, _collate_types)
            else type(t)([fa_convert(s) for s in t]) if isinstance(t, Sequence)
            else default_convert(t))

In [ ]:
t0 = array([1,2])
t = [t0,(t0,t0)]

test_eq(fa_convert(t), default_convert(t))
test_eq(L(fa_convert(t)).map(type), [Tensor,tuple])

In [ ]:
#export
class SkipItemException(Exception): pass

A replacement for PyTorch dataloader. Why replace? Biggest reason: hooks to do what Jeremy wanted to do not available. PyTorch dataloader code (Jeremy thinks) is hard to understand, and pieces are tightly coupled and have assumptions about each other. Hard to add things to it or fix issues with it.  

However PyTorch dataloader is based on well-tested rock solid, fast, multi-processing system and fastai still wanted to use that. 

That mp system is pulled out in PyTorch into _MultiProcessingDataLoaderIter and that is used without any rewrite. But to use it fastai had to write a bit of "slightly ugly code". Specifically code in class _FakeLoader. This exists so we can sneak our way into the PyTorch DataLoading system. 

Overriding a method such as create_item can be done two ways (see below how there are two ways RandDL functionality is implemented). One is inheritance where you get to benefit from state as you get that through inheritance. The other is just passing in create_item as another function defined by us. 

Create item if you look in DataLoader code is in the _methods
list of methods. _methods is a special name and the _methods is going to be looked at by funcs_kwargs decorator. That decorator says the kwargs in this __init__ are not unknown but are actually the list provided by _methods. The decorator automatically grabs any kwargs with these names and it is going to replace these methods that we have with the thing that is passed to that kwarg.

Ofcourse, kwargs are bad for discoverability and Tab Completion and documentation coz we don't know what we can pass to kwargs but fastai fixes those problems. So Dataloader(Shift+Tab) works to reveal all the methods  

In [ ]:
#export
@log_args(but='dataset,wif,create_batch,create_batches,create_item,retain,get_idxs,sample,shuffle_fn,do_batch')
@funcs_kwargs
class DataLoader(GetAttr):
    # Whole bunch of callbacks we can define and use with DataLoader
    # eg after_iter runs after all the iterations. before_iter runs before
    # all the iterations, 

    # do_item to get item, do_batch to create batches - calls before batch,  

    # Can replace any of these callbacks.
    # The _noop_methods currently default to noops or no operations
    # We can just use them as callbacks. 
    _noop_methods = 'wif before_iter after_item before_batch after_batch after_iter'.split()
    for o in _noop_methods:
        exec(f"def {o}(self, x=None, *args, **kwargs): return x")
    _methods = _noop_methods + 'create_batches create_item create_batch retain \
        get_idxs sample shuffle_fn do_batch create_batch'.split()
    _default = 'dataset'
    def __init__(self, dataset=None, bs=None, num_workers=0, pin_memory=False, timeout=0, batch_size=None,
                 shuffle=False, drop_last=False, indexed=None, n=None, device=None, **kwargs):
        if batch_size is not None: bs = batch_size # PyTorch compatibility
        assert not (bs is None and drop_last)
        if indexed is None: indexed = dataset is not None and hasattr(dataset,'__getitem__')
        if n is None:
            try: n = len(dataset)
            except TypeError: pass
        store_attr(self, 'dataset,bs,shuffle,drop_last,indexed,n,pin_memory,timeout,device')
        # to parallelize things automatically - see in _wif code above, setting
        # offset and number of workers and then use it below in the sample 
        # to ensure each process handles contiguous set of things but different
        # from each other process. See sample code below.
        self.rng,self.nw,self.offs = random.Random(random.randint(0,2**32-1)),1,0
        self.fake_l = _FakeLoader(self, pin_memory, num_workers, timeout)

    def __len__(self):
        if self.n is None: raise TypeError
        if self.bs is None: return self.n
        return self.n//self.bs + (0 if self.drop_last or self.n%self.bs==0 else 1)

    def get_idxs(self):
        # if index=True, idxs is infinite count of integers
        # if NOT indexed idxs returns infinite list of Nones
        # so when you shuffle you just get infinite list of Nones
        # in shuffled order

        # Inf class helps to create two types of Infinite Lists
        # Inf.count and Inf.nones. Covered later but pretty important to use 
        # functional programming like paradigm 
        idxs = Inf.count if self.indexed else Inf.nones
        if self.n is not None: idxs = list(itertools.islice(idxs, self.n))
        if self.shuffle: idxs = self.shuffle_fn(idxs)
        return idxs

    def sample(self):
        idxs = self.get_idxs()
        # below is to ensure that each process handles a contiguous set of things
        # which is different to each other process.
        return (b for i,b in enumerate(idxs) if i//(self.bs or 1)%self.nw==self.offs)

    def __iter__(self):
        self.randomize()
        self.before_iter()
        # ugly code below needed for the PyTorch mp dataloader
        # create object of type _FakeLoader which is self.fake_l in init
        # Needed coz PyTorch assumes that it is working with an object
        # which has the list of things specified in _FakeLoader in it
        # So needed to create object with those things with expected values
        # in _FakeLoader. Calling __iter__ on that object is where we pass it
        # back to our DataLoader. This results in tying our DataLoader to
        # some private things in PyTorch like _auto_collation etc on list in
        # _FakeLoader
        for b in _loaders[self.fake_l.num_workers==0](self.fake_l):
            if self.device is not None: b = to_device(b, self.device)
            # this after_batch is called finally after getting 
            # done with do_batch lazily hence yield
            yield self.after_batch(b)
        self.after_iter()
        if hasattr(self, 'it'): delattr(self, 'it')

    def create_batches(self, samps):
        self.it = iter(self.dataset) if self.dataset is not None else None
        # go through everything in the sampler - samps
        # map do_item over it
        res = filter(lambda o:o is not None, map(self.do_item, samps))
        # then chunkify which creates batches out of our list (done lazily)
        # and then call do_batch - since lazily use yield/yield from
        yield from map(self.do_batch, self.chunkify(res))

    def new(self, dataset=None, cls=None, **kwargs):
        if dataset is None: dataset = self.dataset
        if cls is None: cls = type(self)
        cur_kwargs = dict(dataset=dataset, num_workers=self.fake_l.num_workers, pin_memory=self.pin_memory, timeout=self.timeout,
                          bs=self.bs, shuffle=self.shuffle, drop_last=self.drop_last, indexed=self.indexed, device=self.device)
        for n in self._methods: cur_kwargs[n] = getattr(self, n)
        return cls(**merge(cur_kwargs, kwargs))

    @property
    def prebatched(self): return self.bs is None
    def do_item(self, s):
        # calls create_item then calls after_item
        try: return self.after_item(self.create_item(s))
        except SkipItemException: return None
    def chunkify(self, b): return b if self.prebatched else chunked(b, self.bs, self.drop_last)
    def shuffle_fn(self, idxs): return self.rng.sample(idxs, len(idxs))
    def randomize(self): self.rng = random.Random(self.rng.randint(0,2**32-1))
    def retain(self, res, b):  return retain_types(res, b[0] if is_listy(b) else b)
    def create_item(self, s):  return next(self.it) if s is None else self.dataset[s]
    # by default uses PyTorch default_collate and default_convert modified here
    # to appropriately use fa_collate, fa_convert depending on whether your batch
    # size is None or not. You could replace these as is done in fastai language
    # model DataLoader and simplifies that DataLoader considerably by inheriting
    # from this DataLoader and using some of these callbacks. 
    def create_batch(self, b): return (fa_collate,fa_convert)[self.prebatched](b)
    # calls before batch, then creates batch, the self.retain is what retains type
    # then finally up above you see it calls after_batch in __iter__
    def do_batch(self, b): return self.retain(self.create_batch(self.before_batch(b)), b)
    def to(self, device): self.device = device
    def one_batch(self):
        if self.n is not None and len(self)==0: raise ValueError(f'This DataLoader does not contain any batches')
        with self.fake_l.no_multiproc(): res = first(self)
        if hasattr(self, 'it'): delattr(self, 'it')
        return res

In [ ]:
'''
Signature: chunked(it, cs, drop_last=False)
Source:   
def chunked(it, cs, drop_last=False):
    if not isinstance(it, Iterator): it = iter(it)
    while True:
        res = list(itertools.islice(it, cs))
        if res and (len(res)==cs or not drop_last): yield res
        if len(res)<cs: return
File:      /usr/local/lib/python3.6/dist-packages/fastcore/utils.py
'''
chunked??

Override `item` and use the default infinite sampler to get a stream of unknown length (`stop()` when you want to stop the stream).

In [ ]:
'''
Signature: stop(e=<class 'StopIteration'>)
Source:   
def stop(e=StopIteration):
    "Raises exception `e` (by default `StopException`) even if in an expression"
    raise e
File:      /usr/local/lib/python3.6/dist-packages/fastcore/utils.py
'''
stop??

In [ ]:
# Here is an example subclass of DataLoader that overrides create_item
# Normally create_item grabs the i'th item of the dataset assuming that we have some
# sample that we want. 

# Here we override it to return some random number 
# stop raises a StopIteration exception
# In python Generators/Iterators raise this exception to indicate Done.

# Can use this for something reading Network Input Stream for eg. until it gets
# some STOP code. Can create STREAMING DATALOADERs in this way
 
class RandDL(DataLoader):
    def create_item(self, s):
        r = random.random()
        return r if r<0.95 else stop()

L(RandDL())

(#10) [0.12202089395945215,0.2734062294718411,0.40389921044850496,0.08418430973218571,0.8724985568657233,0.4350469849794909,0.8034439090204336,0.13666965035052892,0.6575673348847532,0.40757546504289943]

In [ ]:
L(RandDL(bs=4, drop_last=True)).map(len)

(#12) [4,4,4,4,4,4,4,4,4,4...]

When num_workers=4 this creates 4 streaming data loaders. You end up with more batches than the 0 numworkers version above because you have more streaming workers and they are all doin the RandDL job totally independently. 

In [ ]:
dl = RandDL(bs=4, num_workers=4, drop_last=True)
L(dl).map(len)

(#9) [4,4,4,4,4,4,4,4,4]

In [ ]:
test_eq(dl.fake_l.num_workers, 4)
with dl.fake_l.no_multiproc(): 
    test_eq(dl.fake_l.num_workers, 0)
    L(dl).map(len)
test_eq(dl.fake_l.num_workers, 4)

In [ ]:
# Alternate way to create RandDL 
# Create a _rand_item function and pass that into DataLoader as its create_item
# Does exactly same thing. 

# Anywhere you add callbacks or customizability through inheritance you can also 
# do it by passing in a parameter with the name of the method you wish to override.
# Use this a lot if you wish to change some simple function that you wish to change
# overriding is more work and do not need to understand oo and inheritance etc
# Important limitation of doing it this way - you dont get to use any state
# because you do not get passed in self. If you care about state then you need
# to use first method of inheritance, if you dont then use this way to do it.

def _rand_item(s):
    r = random.random()
    return r if r<0.95 else stop()

L(DataLoader(create_item=_rand_item))

(#3) [0.7845215025498629,0.4475754830394927,0.10627255598425778]

If you don't set `bs`, then `dataset` is assumed to provide an iterator or a `__getitem__` that returns a batch.

If you do not provide bs, then fastai dataloader does not do any batching. Built into PyTorch 1.2 dataloader as well. So idea of bs = None. 

In [ ]:
ds1 = DataLoader(letters)
# No batches created - so listifying DataLoader of letters yields letters back
test_eq(L(ds1), letters)
test_eq(len(ds1), 26)

# DataLoader can be shuffled and test_shuffled checks that the two arguments
# contains all same elements in different orders
test_shuffled(L(DataLoader(letters, shuffle=True)), letters)

# Possible for Dataset to NOT have __getitem__ ie dataset is some kind of 
# infinite stream like a network, or from a GINORMOUS file system, 
# support for NON-INDEXABLE or ITERABLE datasets

# By default fastai DataLoader checks whether what is passed in like letters
# has a __getitem__ and if it DOES NOT, it will treat as ITERABLE, if it DOES
# will treat as INDEXED. Can be overridden with indexed=False implying ITERABLE
# return same thing as before but is doing so by CALLING NEXT rather than
# __getitem__. Useful for GINORMOUS Datasets, 
ds1 = DataLoader(letters, indexed=False)
test_eq(L(ds1), letters)
test_eq(len(ds1), 26)

t2 = L(tensor([0,1,2]),tensor([3,4,5]))
ds2 = DataLoader(t2)
test_eq_type(L(ds2), t2)

# Notice DataLoader converts the arrays to Tensors
t3 = L(array([0,1,2]),array([3,4,5]))
ds3 = DataLoader(t3)
test_eq_type(L(ds3), t3.map(tensor))

ds4 = DataLoader(t3, create_batch=noop, after_iter=lambda: setattr(t3, 'f', 1))
test_eq_type(L(ds4), t3)
test_eq(t3.f, 1)

If you do set `bs`, then `dataset` is assumed to provide an iterator or a `__getitem__` that returns a single item of a batch.

In [ ]:
# Joins each batch together with no separator, then join each set of batches with a
# space between them and do that for two epoch ie two go arounds of all batches.
def twoepochs(d): return ' '.join(''.join(list(o)) for _ in range(2) for o in d)

Can use DataLoader like a PyTorch dataloader. Recall that Dataset is anything that has a length and can index into so letters, a list of letters defined previously is a dataset. num_workers is how many multiprocessing workers to use.

In [ ]:
ds1 = DataLoader(letters, bs=4, drop_last=True, num_workers=0)
test_eq(twoepochs(ds1), 'abcd efgh ijkl mnop qrst uvwx abcd efgh ijkl mnop qrst uvwx')

ds1 = DataLoader(letters,4,num_workers=2)
test_eq(twoepochs(ds1), 'abcd efgh ijkl mnop qrst uvwx yz abcd efgh ijkl mnop qrst uvwx yz')

# can pass in things that can be turned into Tensors like ints in range and like
# PyTorch dataloader it will turn those into batches of Tensors. 
# Note use of test_eq_type checks value and types are same.

ds1 = DataLoader(range(12), bs=4, num_workers=3)
test_eq_type(L(ds1), L(tensor([0,1,2,3]),tensor([4,5,6,7]),tensor([8,9,10,11])))

# HOOKS THAT YOU CAN ADD IN TO FASTAI DataLoader

# One hook is after_iter. It will run at the end of each iteration. 
# Here t3 is just some tensor and it will set t3.f to something so that
# after you run this t3.f has value 2. So you can add code that runs after an 
# iteration 
ds1 = DataLoader([str(i) for i in range(11)], bs=4, after_iter=lambda: setattr(t3, 'f', 2))
test_eq_type(L(ds1), L(['0','1','2','3'],['4','5','6','7'],['8','9','10']))
test_eq(t3.f, 2)

# Can pass not just a Dataset but a generator to a dataloader and it will work 
# fine as well. 
it = iter(DataLoader(map(noop,range(20)), bs=4, num_workers=1))
test_eq_type([next(it) for _ in range(3)], [tensor([0,1,2,3]),tensor([4,5,6,7]),tensor([8,9,10,11])])

In [ ]:
# When you have multiple workers then ensuring everything processed
# comes back in the correct order 

# Here a dataloader that simply returns an item from a list. Added a random
# bit of sleep each time. 

# The tests check that even if you have multiple workers that we get back
# the results in the correct order and %time tests indicate that as you increase
# num_workers the tests do run quickly.
class SleepyDL(list):
    def __getitem__(self,i):
        time.sleep(random.random()/50)
        return super().__getitem__(i)

t = SleepyDL(letters)

%time test_eq(DataLoader(t, num_workers=0), letters)
%time test_eq(DataLoader(t, num_workers=2), letters)
%time test_eq(DataLoader(t, num_workers=4), letters)

dl = DataLoader(t, shuffle=True, num_workers=1)
test_shuffled(L(dl), letters)
test_shuffled(L(dl), L(dl))

CPU times: user 6.14 ms, sys: 163 µs, total: 6.31 ms
Wall time: 269 ms
CPU times: user 9.34 ms, sys: 17.8 ms, total: 27.1 ms
Wall time: 180 ms
CPU times: user 16.2 ms, sys: 26.1 ms, total: 42.3 ms
Wall time: 125 ms


In [ ]:
# Example of using only __iter__ vs __getitem__ 
# So only able to iterate, 

# Example of working with iterative only Q with variable latency (could be 
# something streaming over a Network),

# Test checks that we get back the right thing and sleepy Q only has __iter__
# not __getitem__ so no indexing so no guarantee of what order things will
# come back in so we end up with something that is shuffled.

# Also note that if index is False, shuffle does NOT do anything.
# Implementation in DataLoader (see above) if index is False you just
# get Infinite list of Nones in shuffled order so it does not matter.
class SleepyQueue():
    "Simulate a queue with varying latency"
    def __init__(self, q): self.q=q
    def __iter__(self):
        while True:
            time.sleep(random.random()/100)
            try: yield self.q.get_nowait()
            except queues.Empty: return

q = Queue()
for o in range(30): q.put(o)
it = SleepyQueue(q)

%time test_shuffled(L(DataLoader(it, num_workers=4)), range(30))

CPU times: user 14.2 ms, sys: 28.6 ms, total: 42.8 ms
Wall time: 132 ms


In [ ]:
class A(TensorBase): pass

for nw in (0,2):
    t = A(tensor([1,2]))
    dl = DataLoader([t,t,t,t,t,t,t,t], bs=4, num_workers=nw)
    b = first(dl)
    test_eq(type(b), A)

    t = (A(tensor([1,2])),)
    dl = DataLoader([t,t,t,t,t,t,t,t], bs=4, num_workers=nw)
    b = first(dl)
    test_eq(type(b[0]), A)

In [ ]:
class A(TensorBase): pass
t = A(tensor(1,2))

tdl = DataLoader([t,t,t,t,t,t,t,t], bs=4, num_workers=2, after_batch=to_device)
b = first(tdl)
test_eq(type(b), A)

# Unknown attributes are delegated to `dataset`
test_eq(tdl.pop(), tensor(1,2))

## Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_torch_core.ipynb.
Converted 01_layers.ipynb.
Converted 02_data.load.ipynb.
Converted 03_data.core.ipynb.
Converted 04_data.external.ipynb.
Converted 05_data.transforms.ipynb.
Converted 06_data.block.ipynb.
Converted 07_vision.core.ipynb.
Converted 08_vision.data.ipynb.
Converted 09_vision.augment.ipynb.
Converted 09b_vision.utils.ipynb.
Converted 09c_vision.widgets.ipynb.
Converted 10_tutorial.pets.ipynb.
Converted 11_vision.models.xresnet.ipynb.
Converted 12_optimizer.ipynb.
Converted 13_callback.core.ipynb.
Converted 13a_learner.ipynb.
Converted 13b_metrics.ipynb.
Converted 14_callback.schedule.ipynb.
Converted 14a_callback.data.ipynb.
Converted 15_callback.hook.ipynb.
Converted 15a_vision.models.unet.ipynb.
Converted 16_callback.progress.ipynb.
Converted 17_callback.tracker.ipynb.
Converted 18_callback.fp16.ipynb.
Converted 18a_callback.training.ipynb.
Converted 19_callback.mixup.ipynb.
Converted 20_interpret.ipynb.
Converted 20a_distributed.ipynb.
Converted 21_vision.l